In [ ]:
!nvidia-smi

Wed Sep 20 03:14:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install torch torchvision torchaudio
# !pip install transformers torch -U
# !pip install accelerate -U
# !pip install git+https://github.com/huggingface/accelerate
# !pip install transformers[torch]

In [ ]:
!pip install --upgrade accelerate -q
!pip install --upgrade torch -q
!pip install transformers -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import os

os.chdir("/content/drive/MyDrive/Document Tagging")

In [ ]:
!pwd

/content/drive/MyDrive/Document Tagging


In [ ]:
import string
import re
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd


custom_words = ['is', 'are', 'have', 'can', 'will', 'do', 'i', 'you', 'he', 'she', 'it', 'we',
                'they', 'me', 'him', 'her', 'us', 'them', 'a', 'an', 'the', 'in', 'on', 'at', 'with', 'by',
                'for', 'of', 'to', 'from', 'and', 'or', 'but', 'so', 'yet', 'nor', 'null', 'nan', 'no', 'not',
                'not', 'very', 'always', 'never', 'too', 'often', 'sometimes', 'rarely', 'frequently', 'quickly', 'slowly',
                'easily', 'hardly', 'nearly', 'completely', 'always', 'never', 'generally', 'specifically', 'approximately', 'constantly',
                'occasionally', 'regularly', 'precisely', 'thoroughly']


nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
wiki_data = pd.read_csv("auto_tagging_data.csv")
# wiki_data = wiki_data.head(5)

# wiki_data = wiki_data[0:10000]

In [ ]:

wiki_data.shape


(76365, 2)

In [ ]:
# handle the data: missing values & drop duplicates


def handle_data(data):
  data.fillna(method='ffill', inplace=True) # fill the null values usng forward-fill method
  wiki_data.drop_duplicates(inplace=True) # remove duplicates
  return data


data = handle_data(data=wiki_data)
data.head()


,Title,Tags
0,The Two Cultures: statistics vs. machine learn...,['machine-learning']
1,Forecasting demographic census,['forecasting']
2,Bayesian and frequentist reasoning in plain En...,['bayesian']
3,What is the meaning of p values and t values i...,['hypothesis-testing' 't-test' 'p-value' 'inte...
4,Examples for teaching: Correlation does not me...,['correlation']


In [ ]:
# Separate X & Y:

def separate_x_y(data, y_col_name):
  Y = data[y_col_name]
  X = data.drop(y_col_name, axis=1)
  X = [item for sublist in X.values.tolist() for item in sublist ]


  return X, Y


X, Y = separate_x_y(data=wiki_data, y_col_name="Tags")


In [ ]:
# Text-Preprocessing:

def text_preprocessing(data):
  sentences = data
  preprocessed_sentences = []

  for sentence in sentences:
    # lowering the sentences:
    text = sentence.lower()

    # remove punctuation:
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # remove the numbers:
    text = re.sub(r'\d+', '', text)

    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)

    # word tokenization:
    tokens = word_tokenize(text)

    # remove the stop of words:
    words = [word for word in tokens if word not in custom_words]


    # get the pre-processed sentences:
    preprocessed_sentence = ' '.join(words)
    preprocessed_sentences.append(preprocessed_sentence)

  return preprocessed_sentences


In [ ]:
titles = text_preprocessing(X)
Y__ = text_preprocessing(Y)

In [ ]:
titles[0:5]

['two cultures statistics vs machine learning',
 'forecasting demographic census',
 'bayesian frequentist reasoning plain english',
 'what meaning p values t values statistical tests',
 'examples teaching correlation does mean causation']

In [ ]:
Y__[0:5]

['machinelearning',
 'forecasting',
 'bayesian',
 'hypothesistesting ttest pvalue interpretation',
 'correlation']

In [ ]:
y_data_series = pd.Series(Y__)

# Split each string into a list of tags
tags_data = list(y_data_series.str.split())

In [ ]:
tags_data[0:3]

[['machinelearning'], ['forecasting'], ['bayesian']]

In [ ]:
tags = tags_data

In [ ]:
dd = {"Title":titles, "Tags":tags}
data = pd.DataFrame(dd)
data


,Title,Tags
0,two cultures statistics vs machine learning,[machinelearning]
1,forecasting demographic census,[forecasting]
2,bayesian frequentist reasoning plain english,[bayesian]
3,what meaning p values t values statistical tests,"[hypothesistesting, ttest, pvalue, interpretat..."
4,examples teaching correlation does mean causation,[correlation]
...,...,...
76351,interpretation global test pvalue interaction ...,"[logistic, categoricaldata, interaction, inter..."
76352,testing my linear model,"[linearmodel, goodnessoffit]"
76353,how know simple validation result statisticall...,"[machinelearning, hypothesistesting, statistic..."
76354,kendall conditional independence test statistic,"[hypothesistesting, selfstudy]"


In [ ]:
unique_tags = set(tag for tag_list in tags for tag in tag_list)
tag2id = {tag: idx for idx, tag in enumerate(unique_tags)}

tag2id


In [ ]:
len(tag2id)

100

In [ ]:
data['Tags'] = data['Tags'].apply(lambda tags: [tag2id[tag] for tag in tags if tag in tag2id])


In [ ]:
data.head()


,Title,Tags
0,two cultures statistics vs machine learning,[49]
1,forecasting demographic census,[48]
2,bayesian frequentist reasoning plain english,[32]
3,what meaning p values t values statistical tests,"[74, 82, 52, 68]"
4,examples teaching correlation does mean causation,[38]


In [ ]:

# from transformers import AutoModelForTokenClassification, AutoTokenizer
# import torch

# # Load tokenizer and model
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model_name = "bert-base-uncased"


# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(tag2id)).to(device)


In [ ]:


from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load tokenizer and model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "QCRI/bert-base-multilingual-cased-pos-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(tag2id), ignore_mismatched_sizes=True).to(device)




Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([46, 768]) in the checkpoint and torch.Size([100, 768]) in the model 

In [ ]:

tokenized_titles = tokenizer(titles, padding=True, truncation=True, return_tensors='pt')


In [ ]:

input_ids = tokenized_titles['input_ids']
attention_mask = tokenized_titles['attention_mask']


In [ ]:
input_ids

tensor([[  101, 10551, 36782,  ...,     0,     0,     0],
        [  101, 10142, 33588,  ...,     0,     0,     0],
        [  101, 16184, 74946,  ...,     0,     0,     0],
        ...,
        [  101, 14796, 21852,  ...,     0,     0,     0],
        [  101, 67680, 14555,  ...,     0,     0,     0],
        [  101, 75382, 10310,  ...,     0,     0,     0]])

In [ ]:
len(input_ids[10]), len(input_ids[100])


(42, 42)

In [ ]:
desired_length = len(input_ids[10])

# Pad the lists with -100 to make them all the same length
tags_data_padded = [tag_list + [-100] * (desired_length - len(tag_list)) for tag_list in data['Tags']]

# Convert the padded list of lists into a PyTorch tensor
tag_data_tensor = torch.tensor(tags_data_padded)
tag_data_tensor

tensor([[  49, -100, -100,  ..., -100, -100, -100],
        [  48, -100, -100,  ..., -100, -100, -100],
        [  32, -100, -100,  ..., -100, -100, -100],
        ...,
        [  49,   74,   53,  ..., -100, -100, -100],
        [  74,   58, -100,  ..., -100, -100, -100],
        [  74, -100, -100,  ..., -100, -100, -100]])

In [ ]:

len(tag_data_tensor[100])


42

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

In [ ]:

X_input_id_tensor  = [torch.tensor(seq, dtype=torch.long) for seq in input_ids]
X_attn_mask_tensor = [torch.tensor(seq, dtype=torch.long) for seq in attention_mask]


In [ ]:
# X_input_id_tensor


In [ ]:

Y_tensor = [torch.tensor(seq, dtype=torch.long) for seq in tag_data_tensor]
# Y_tensor


In [ ]:

X_train, X_temp, Y_train, Y_temp, attn_mask_X_train, attn_mask_X_temp = train_test_split(
    X_input_id_tensor,
    Y_tensor,
    X_attn_mask_tensor,
    test_size=0.5,
    random_state=0
)

X_valid, X_test, Y_valid, Y_test, attn_mask_X_valid, attn_mask_X_test = train_test_split(
    X_temp,
    Y_temp,
    attn_mask_X_temp,
    test_size=0.2,
    random_state=0
)


In [ ]:
print(f"X_train: \t {len(X_train)}")
print(f"X_test: \t {len(X_test)}")
print(f"X_valid: \t {len(X_valid)}")


X_train: 	 38178
X_test: 	 7636
X_valid: 	 30542


In [ ]:

batch = len(tag_data_tensor[100])

# Filter examples with empty labels or labels that don't match batch size
def filter_examples(input_ids, attention_mask, labels, batch_size):
    filtered_input_ids = []
    filtered_attention_mask = []
    filtered_labels = []

    for i in range(len(input_ids)):
        if len(labels[i]) == batch_size:
            filtered_input_ids.append(input_ids[i])
            filtered_attention_mask.append(attention_mask[i])
            filtered_labels.append(labels[i])
        else:
            print(f"Example {i} has incorrect label size: {len(labels[i])}")

    return filtered_input_ids, filtered_attention_mask, filtered_labels


X_train, attn_mask_X_train, Y_train = filter_examples(X_train, attn_mask_X_train, Y_train, batch_size=batch)
X_valid, attn_mask_X_valid, Y_valid = filter_examples(X_valid, attn_mask_X_valid, Y_valid, batch_size=batch)
X_test, attn_mask_X_test, Y_test = filter_examples(X_test, attn_mask_X_test, Y_test, batch_size=batch)

In [ ]:


from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }

# Load your preprocessed data
train_dataset = CustomDataset(X_train, attn_mask_X_train, Y_train)
test_dataset = CustomDataset(X_test, attn_mask_X_test, Y_test)
valid_dataset = CustomDataset(X_valid, attn_mask_X_valid, Y_valid)





In [ ]:
train_dataset[0]

{'input_ids': tensor([  101, 43977, 13213, 32095,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([  18,   15, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100])}

In [ ]:

def collate_fn(features):
    input_ids = torch.nn.utils.rnn.pad_sequence([feature['input_ids'] for feature in features], batch_first=True)
    attention_mask = torch.nn.utils.rnn.pad_sequence([feature['attention_mask'] for feature in features], batch_first=True)
    labels = torch.nn.utils.rnn.pad_sequence([feature['labels'] for feature in features], batch_first=True)
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


d = collate_fn(test_dataset)
d

{'input_ids': tensor([[  101, 29152, 18873,  ...,     0,     0,     0],
         [  101, 70204, 11795,  ...,     0,     0,     0],
         [  101, 77239, 10376,  ...,     0,     0,     0],
         ...,
         [  101, 14796, 74723,  ...,     0,     0,     0],
         [  101, 14796, 28840,  ...,     0,     0,     0],
         [  101, 10571, 65211,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[  49, -100, -100,  ..., -100, -100, -100],
         [  94, -100, -100,  ..., -100, -100, -100],
         [  94,   97,   87,  ..., -100, -100, -100],
         ...,
         [  94,   11, -100,  ..., -100, -100, -100],
         [  20,   52, -100,  ..., -100, -100, -100],
         [  74, -100, -100,  ..., -100, -100, -100]])}

In [ ]:
!pip install transformers datasets tokenizers seqeval -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00


In [ ]:

import datasets
metric = datasets.load_metric("seqeval")


In [ ]:

label_list = [key for key, val in tag2id.items()]

def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }



In [ ]:

from transformers import Trainer, TrainingArguments




training_args = TrainingArguments(
        output_dir="output",
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=10,
        weight_decay=0.01,
        save_total_limit=0,
        save_strategy="no"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=collate_fn,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



In [ ]:

import torch
torch.cuda.empty_cache()


In [ ]:

trainer.train()



Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,2.699000,2.632336,0.286496,0.205101,0.239060,0.365479
2,2.458000,2.493336,0.304324,0.221429,0.256342,0.388064
3,2.259600,2.456569,0.317752,0.235634,0.270600,0.395156
4,2.066000,2.475587,0.319705,0.240446,0.274468,0.392721
5,1.857500,2.523153,0.318483,0.241757,0.274866,0.387279
6,1.721500,2.592621,0.311300,0.235070,0.267867,0.383504
7,1.519500,2.658537,0.311702,0.240230,0.271339,0.378732
8,1.375700,2.743752,0.310626,0.240844,0.271320,0.374352
9,1.242300,2.801788,0.310434,0.242056,0.272014,0.373568
10,1.184700,2.834649,0.311015,0.244346,0.273679,0.373372


TrainOutput(global_step=47730, training_loss=1.874109653974014, metrics={'train_runtime': 7624.9654, 'train_samples_per_second': 50.07, 'train_steps_per_second': 6.26, 'total_flos': 8190513717624000.0, 'train_loss': 1.874109653974014, 'epoch': 10.0})

In [ ]:

trainer.evaluate()


{'eval_loss': 2.834648609161377,
 'eval_precision': 0.3110148906959552,
 'eval_recall': 0.24434561837279922,
 'eval_f1': 0.27367853691175376,
 'eval_accuracy': 0.3733719010965665,
 'eval_runtime': 106.1326,
 'eval_samples_per_second': 287.772,
 'eval_steps_per_second': 35.974,
 'epoch': 10.0}

In [ ]:

model.save_pretrained("my_model")
tokenizer.save_pretrained("tokenizer")


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:

label2id = {key:str(val)  for key, val in tag2id.items()}
id2label = {str(val):key  for key, val in tag2id.items()}


In [ ]:

import json

config = json.load(open("my_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("my_model/config.json","w"))


In [ ]:

model_fine_tuned = AutoModelForTokenClassification.from_pretrained("my_model")


In [ ]:
from transformers import pipeline

# ner
nlp = pipeline(task='ner', model=model_fine_tuned, tokenizer=tokenizer)


example = "Statistics vs. machine learning"

ner_results = nlp(example)

print(ner_results)




[{'entity': 'references', 'score': 0.3105938, 'index': 1, 'word': 'Statistics', 'start': 0, 'end': 10}, {'entity': 'classification', 'score': 0.15585431, 'index': 2, 'word': 'vs', 'start': 11, 'end': 13}, {'entity': 'machinelearning', 'score': 0.28311414, 'index': 3, 'word': '.', 'start': 13, 'end': 14}, {'entity': 'svm', 'score': 0.15134172, 'index': 4, 'word': 'machine', 'start': 15, 'end': 22}, {'entity': 'references', 'score': 0.19858816, 'index': 5, 'word': 'learning', 'start': 23, 'end': 31}]


In [ ]:
for dct in ner_results:
  print(dct['entity'])

references
classification
machinelearning
svm
references
